In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import glob
import geopandas as gpd
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt


from darts.models import TFTModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, MonthlyMilkDataset, ElectricityDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

In [3]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')


meteo_input = ['prcp_e5l',  't_max_e5l', 't_min_e5l']
# q_mm_day or lvl_sm
hydro_target = 'lvl_sm'


Using device: cuda

NVIDIA GeForce RTX 3080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
if (hydro_target == 'lvl_mbs') | (hydro_target == 'lvl_sm'):
    static_parameters = ['for_pc_sse', 'crp_pc_sse',
                         'inu_pc_ult', 'ire_pc_sse',
                         'lka_pc_use', 'prm_pc_sse',
                         'pst_pc_sse', 'cly_pc_sav',
                         'slt_pc_sav', 'snd_pc_sav',
                         'kar_pc_sse', 'urb_pc_sse',
                         'gwt_cm_sav', 'lkv_mc_usu',
                         'rev_mc_usu', 'sgr_dk_sav',
                         'slp_dg_sav', 'ws_area',
                         'ele_mt_sav', 'height_bs']
    nc_variable = 'nc_all_h'
else:
    static_parameters = ['for_pc_sse', 'crp_pc_sse',
                         'inu_pc_ult', 'ire_pc_sse',
                         'lka_pc_use', 'prm_pc_sse',
                         'pst_pc_sse', 'cly_pc_sav',
                         'slt_pc_sav', 'snd_pc_sav',
                         'kar_pc_sse', 'urb_pc_sse',
                         'gwt_cm_sav', 'lkv_mc_usu',
                         'rev_mc_usu', 'sgr_dk_sav',
                         'slp_dg_sav', 'ws_area',
                         'ele_mt_sav']
    nc_variable = 'nc_all_q'

ws_file = gpd.read_file('../geo_data/great_db/geometry/russia_ws.gpkg')
ws_file = ws_file.set_index('gauge_id')

In [5]:
with_static = True
index_col = 'gauge_id'
static_path = '../geo_data/attributes/static_with_height.csv'
nc_files = glob.glob(f'../geo_data/great_db/{nc_variable}/*.nc')[:10]
area_index = ws_file.index

In [6]:
def file_checker(file_path: str,
                 meteo_predictors: list,
                 hydro_target: str):
    nan_vals = xr.open_dataset(
        file_path).to_dataframe()[
            [hydro_target,
             *meteo_predictors]].isna().sum().sum()

    condition = nan_vals != 0

    return condition

In [7]:
if with_static:
    static_attributes = pd.read_csv(static_path,
                                    index_col=index_col)
    static_attributes.index = static_attributes.index.astype(str)
    static_attributes = static_attributes.replace(np.NaN, 0.)
else:
    static_attributes = None

res_file = list()

for file_path in nc_files[:10]:
    gauge_id = file_path.split('/')[-1][:-3]
    try:
        if isinstance(static_attributes, pd.DataFrame):
            static_attributes = static_attributes[[*static_parameters]]
            gauge_static = static_attributes.loc[[gauge_id], :]
        else:
            gauge_static = None
    except KeyError:
        print(f'No data for {gauge_id} !')
        continue
    cond = file_checker(file_path=file_path,
                        meteo_predictors=meteo_input,
                        hydro_target=hydro_target)
    if cond:
        continue
    if gauge_id not in area_index:
        continue
    file = xr.open_dataset(file_path).to_dataframe()
    file = file.reset_index()
    if isinstance(gauge_static, pd.DataFrame):
            for col in gauge_static.columns:
                file[col] = gauge_static.loc[gauge_id, col]
    res_file.append(file)

file = pd.concat(res_file, axis=0)
file = file.reset_index(drop=True)

In [8]:
scaler = StandardScaler()
file[[hydro_target, *meteo_input, *static_parameters]] = scaler.fit_transform(
    file[[hydro_target, *meteo_input, *static_parameters]])

cov_time_series = TimeSeries.from_group_dataframe(
    df=file,
    time_col='date',
    group_cols='gauge_id',
    static_cols=static_parameters,
    value_cols=meteo_input)
cov_time_series = [series.with_static_covariates(
    series.static_covariates[static_parameters])
                   for series in cov_time_series]
target_time_series = TimeSeries.from_group_dataframe(
    df=file,
    time_col='date',
    group_cols='gauge_id',
    static_cols=static_parameters,
    value_cols=hydro_target)
target_time_series = [series.with_static_covariates(
    series.static_covariates[static_parameters])
                   for series in target_time_series]

In [10]:
cov_train_val_dict = dict()
target_train_val_dict = dict()

train_start = '2008-01-01'
train_end = '2016-12-31'

val_start = '2017-01-01'
val_end = '2018-12-31'

pred_start = '2018-01-01'
pred_end = '2020-12-31'

# train period
cov_train_val_dict['train'] = list()
for g in cov_time_series:
    cov_train_val_dict['train'].append(g.split_after(
        pd.Timestamp(train_end))[0].astype(np.float32))
# train period
target_train_val_dict['train'] = list()
for g in target_time_series:
    target_train_val_dict['train'].append(g.split_after(
        pd.Timestamp(train_end))[0].astype(np.float32))

# val period
cov_train_val_dict['val'] = list()
for g in cov_time_series:
    cov_train_val_dict['val'].append(g.split_before(
        pd.Timestamp(val_start))[1].split_after(
            pd.Timestamp(val_end))[0].astype(np.float32))
# val period
target_train_val_dict['val'] = list()
for g in target_time_series:
    target_train_val_dict['val'].append(g.split_before(
        pd.Timestamp(val_start))[1].split_after(
            pd.Timestamp(val_end))[0].astype(np.float32))

# test period
cov_train_val_dict['test'] = list()
for g in cov_time_series:
    cov_train_val_dict['test'].append(g.split_before(
        pd.Timestamp(pred_start))[1].split_after(
            pd.Timestamp(pred_end))[0].astype(np.float32))
# test period
target_train_val_dict['test'] = list()
for g in target_time_series:
    test_split = g.split_before(
        pd.Timestamp(pred_start))[1].split_after(
            pd.Timestamp(pred_end))[0].astype(np.float32)
    target_train_val_dict['test'].append(test_split)

In [11]:
# default quantiles for QuantileRegression
quantiles = [
    0.01,
    0.05,
    0.1,
    0.15,
    0.2,
    0.25,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.75,
    0.8,
    0.85,
    0.9,
    0.95,
    0.99,
]
input_chunk_length = 365
forecast_horizon = 7
my_model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=forecast_horizon,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.4,
    batch_size=128,
    n_epochs=10,
    add_relative_index=False,
    add_encoders=None,
    likelihood=QuantileRegression(
        quantiles=quantiles
    ),  # QuantileRegression is set per default
    # loss_fn=MSELoss(),
    random_state=42,
)

In [12]:
my_model.fit(target_train_val_dict['train'],
             future_covariates=cov_train_val_dict['train'],
             verbose=True)

A Jupyter Widget

TFTModel(hidden_size=64, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.4, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=QuantileRegression(quantiles: Optional[List[float]] = None), norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=365, output_chunk_length=7, batch_size=128, n_epochs=10, add_encoders=None, random_state=42)

In [19]:
# before starting, we define some constants
num_samples = 24

figsize = (9, 6)
lowest_q, low_q, high_q, highest_q = 0.01, 0.1, 0.9, 0.99
label_q_outer = f"{int(lowest_q * 100)}-{int(highest_q * 100)}th percentiles"
label_q_inner = f"{int(low_q * 100)}-{int(high_q * 100)}th percentiles"

In [ ]:
my_model.pre

In [32]:
my_model.historical_forecasts(num_samples=num_samples,
                              series=target_train_val_dict['test'],
                              past_covariates=cov_train_val_dict['test'],
                              future_covariates=cov_train_val_dict['test'])

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [31]:
my_model.predict(n=7,
                 num_samples=num_samples,
                 series=target_train_val_dict['test'],
                 past_covariates=cov_train_val_dict['test'],
                 future_covariates=cov_train_val_dict['test'])

ValueError: For the given forecasting horizon `n=7`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n <= output_chunk_length` the future covariates must end at time step `2021-01-07 00:00:00`, whereas now they end at time step `2020-12-31 00:00:00`.

In [17]:
def eval_model(model, n, actual_series, val_series):
    pred_series = model.predict(n=n, num_samples=10)

    # plot actual series
    plt.figure(figsize=figsize)
    actual_series[: pred_series.end_time()].plot(label="actual")

    # plot prediction with quantile ranges
    pred_series.plot(
        low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer
    )
    pred_series.plot(low_quantile=low_q, high_quantile=high_q, label=label_q_inner)

    plt.title("MAPE: {:.2f}%".format(mape(val_series, pred_series)))
    plt.legend()


eval_model(my_model, 7, target_time_series, target_train_val_dict['val'])



ValueError: Input `series` must be provided. This is the result either from fitting on multiple series, or from not having fit the model yet.